In [1]:
import re
import numpy as np
from sklearn import preprocessing
import pandas as pd

# Getting Data
train_df= pd.read_csv('training_titanic_x_y_train.csv')
test_df = pd.read_csv('test_titanic_x_test.csv')

data=[train_df,test_df]

# Data Exploration / Analysis
for i in data:
    for j in i.keys():
        if i[j].isnull().any():
            print(j,' Has Null Values')
    print(i.describe())
    print(i.info())


Age  Has Null Values
Cabin  Has Null Values
Embarked  Has Null Values
           Pclass         Age       SibSp       Parch        Fare    Survived
count  668.000000  536.000000  668.000000  668.000000  668.000000  668.000000
mean     2.296407   29.700560    0.528443    0.407186   32.064552    0.402695
std      0.831638   14.240257    1.080327    0.854695   45.320835    0.490808
min      1.000000    0.670000    0.000000    0.000000    0.000000    0.000000
25%      2.000000   21.000000    0.000000    0.000000    7.925000    0.000000
50%      3.000000   29.000000    0.000000    0.000000   14.750000    0.000000
75%      3.000000   38.250000    1.000000    0.000000   31.275000    1.000000
max      3.000000   80.000000    8.000000    6.000000  512.329200    1.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 11 columns):
Pclass      668 non-null int64
Name        668 non-null object
Sex         668 non-null object
Age         536 non-null flo

In [2]:
# Missing Data

# Cabin to Deck
data = [train_df, test_df]
def getDeck(x) :
    return x[0:1]

for dataset in data :
    dataset['Cabin'].fillna('U0', inplace=True)
    dataset['Deck'] = dataset.Cabin.apply(getDeck)
    
# Drop Cabin
train_df = train_df.drop(columns = ['Cabin'])
test_df = test_df.drop(columns = ['Cabin'])

#TicketNo
data = [train_df, test_df]
import re
for dataset in data:
    dataset['TicNO']=dataset.Ticket.str.extract('([0-9]+)$',expand=False)
    #x=[len(str(j)) for j in dataset['TicNO'].unique()]
    #print(set(x))
    
# Drop Ticket 
train_df = train_df.drop(columns=['Ticket'])
test_df = test_df.drop(columns=['Ticket'])


# Age
survived_mean = train_df[train_df.Survived == 1].Age.mean()
not_survived_mean = train_df[train_df.Survived == 0].Age.mean()
test_mean = test_df.Age.mean()

train_df.update(train_df[train_df.Survived==1].Age.fillna(survived_mean))
train_df.update(train_df[train_df.Survived==0].Age.fillna(not_survived_mean))
test_df.Age.fillna(test_mean, inplace=True)


# Embarked
data = [train_df, test_df]
for dataset in data :
    dataset.Embarked.fillna('S', inplace=True)
    
#Name to Title 
data = [train_df, test_df]
for dataset in data:
    dataset['Title']=dataset.Name.str.extract('([A-Za-z]+)\.',expand=False)
    dataset['Title']=dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
    dataset['Title']=dataset['Title'].replace(['Ms'],'Miss')
    dataset['Title']=dataset['Title'].replace(['Mlle'],'Miss')
    dataset['Title']=dataset['Title'].replace(['Mme'],'Mrs')

# Drop Name
train_df = train_df.drop(columns=['Name'])
test_df = test_df.drop(columns=['Name'])

# Getting Survived Column
y_df = train_df.Survived
train_df = train_df.drop(columns = ['Survived'])


In [3]:
data = [train_df, test_df]
for i in data:
    print(i.keys())

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Deck',
       'TicNO', 'Title'],
      dtype='object')
Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Deck',
       'TicNO', 'Title'],
      dtype='object')


In [4]:
# Categorical Data
def setTicNo(x):
    x=len(str(x))
    if x == 1 :
        return 0
    elif x == 3 :
        return 1
    elif x == 4 :
        return 2
    elif x == 5 :
        return 3
    elif x == 6 :
        return 4
    else:
        return 5
    
def setSex(x) :
    if x == 'male' :
        return 1
    else :
        return 0

def setEmbarked(x) :
    if x == 'S' :
        return 0
    elif x == 'C' :
        return 1
    else:
        return 2

def setDeck(x) :
    dic = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'U':7}
    return dic[x]

def if_T_in_Deck(x) :
    if x == 'T' :
        return 'A'
    else :
        return x

train_df.Deck = train_df.Deck.apply(if_T_in_Deck)

def setFare(x):
    if x<=8 and x>=0:
        return 0
    elif x>8 and x<=15 :
        return 1
    elif x>15 and x<=31 :
        return 2
    elif x>31 and x<=99 :
        return 3
    elif x>99 and x<=250 :
        return 4
    elif x>250 :
        return 5
    
def setAge(x):
    if x<=11 and x>=0:
        return 0
    elif x>11 and x<=18 :
        return 1
    elif x>18 and x<=22 :
        return 2
    elif x>22 and x<=27 :
        return 3
    elif x>27 and x<=33 :
        return 4
    elif x>33 and x<=40 :
        return 5
    elif x>40 and x<=66 :
        return 6
    elif x>66 :
        return 7

def setPClass(x):
    if x == 1 :
        return 0
    elif x == 2 :
        return 1
    else:
        return 2
    
def setTitle(x):
    titles={'Mr':0,'Miss':1,'Mrs':2,'Master':3,'Rare':4}
    return titles[x]

def setSibSp(x):
    if x == 8:
        return 6
    else:
        return x
    

data = [train_df, test_df]
for dataset in data :
    dataset['Sex'] = dataset['Sex'].apply(setSex)
    dataset['Embarked'] = dataset['Embarked'].apply(setEmbarked)
    dataset['Deck'] = dataset['Deck'].apply(setDeck)
    dataset['Fare'] = dataset['Fare'].apply(setFare)
    dataset['Age'] = dataset['Age'].apply(setAge)
    dataset['Pclass'] = dataset['Pclass'].apply(setPClass)
    dataset['Title'] = dataset['Title'].apply(setTitle)
    dataset['SibSp'] = dataset['SibSp'].apply(setSibSp)
    dataset['TicNO'] = dataset['TicNO'].apply(setTicNo)

for dataset in data :
    n=len(dataset.columns)
    for i in range(n):
        r=str(dataset.columns[i])+'^2'
        dataset[r]=dataset[dataset.columns[i]]**2
    print(dataset.describe()) 
    print(dataset.info())



           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  668.000000  668.000000  668.000000  668.000000  668.000000  668.000000   
mean     1.296407    0.639222    3.613772    0.513473    0.407186    1.571856   
std      0.831638    0.480586    1.747285    0.986322    0.854695    1.249618   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000    0.000000    3.000000    0.000000    0.000000    0.000000   
50%      2.000000    1.000000    4.000000    0.000000    0.000000    1.000000   
75%      2.000000    1.000000    5.000000    1.000000    0.000000    3.000000   
max      2.000000    1.000000    7.000000    6.000000    6.000000    5.000000   

         Embarked        Deck       TicNO       Title    Pclass^2       Sex^2  \
count  668.000000  668.000000  668.000000  668.000000  668.000000  668.000000   
mean     0.348802    5.949102    3.344311    0.732036    2.371257    0.639222   
std      0.614333    2.0445

In [12]:
from sklearn.linear_model import LogisticRegression
algo = LogisticRegression(C=7.0)
algo.fit(train_df, y_df)
y_pred = algo.predict(test_df)
data = [train_df, test_df]
np.savetxt('titanic_prediction.csv', y_pred, fmt='%1.5f')

C:\Users\Anisha\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
